In [17]:
import numpy as np
import pandas as pd
import datetime


In [47]:
%%time

wind_farms = pd.read_csv("../data/wind_farms_with_clusters.csv")
hourly_wind_data = pd.read_csv("../data/hourly_data.csv")

Wall time: 7.56 s


In [48]:
wind_farms

,Unnamed: 0,farm_name,zone,city,turbines,latitude,longitude,kmeans_labels
0,0,A Capelada I,La Coruna (Galicia),"Cariño, Cedeira, Ortigueira",50,43.717,-7.937,2
1,1,A Capelada II,La Coruna (Galicia),"Cariño, Cedeira, Ortigueira",45,43.687,-7.960,2
2,2,A Pastoriza,Lugo (Galicia),"A Pastoriza, Mondoñedo, Riotorto",10,43.374,-7.327,2
3,3,A Ruña,La Coruna (Galicia),"Muros, Mazaricos",41,42.940,-8.990,2
4,6,Abuela Santa Ana,Albacete (Castilla-La Mancha),Pozo Llorente,25,39.053,-1.502,4
...,...,...,...,...,...,...,...,...
995,1195,Yerga II,Logroño (La Rioja),"Alfaro, Autol",36,42.144,-1.956,3
996,1196,Zarzuela II,Cadiz (Andalucía),Tarifa,8,36.010,-5.600,0
997,1197,Zas,La Coruna (Galicia),"Zas, Santa Comba",90,43.100,-8.870,2
998,1198,Zorraquín,Soria (Castilla y Leon),"Gomyo, Villaciervos",6,41.760,-2.630,3


In [49]:
hourly_wind_data

,datetime,value,A Capelada I,A Capelada II,A Pastoriza,A Ruña,Ablitas,Abote,Abuela Santa Ana,Abuela Santa Ana Modificación,...,Xiabre Ampliación,Xiabre I,Xiabre II,Xunqueira,Yerga I,Yerga II,Zarzuela II,Zas,Zorraquín,Zorreras
0,2016-09-01 00:00:00,6373.0,5.37,5.37,4.41,5.19,5.38,11.05,3.18,3.18,...,5.35,5.43,5.43,4.12,5.38,5.38,5.71,5.19,2.62,5.71
1,2016-09-01 01:00:00,5976.0,5.29,5.29,4.31,5.11,5.29,11.04,3.00,3.00,...,5.03,5.35,5.35,4.06,5.29,5.29,6.51,5.11,2.49,6.51
2,2016-09-01 02:00:00,5499.0,5.31,5.31,4.24,5.05,4.95,11.13,2.94,2.94,...,4.62,5.03,5.03,4.04,4.95,4.95,7.29,5.05,2.35,7.29
3,2016-09-01 03:00:00,5031.0,5.27,5.27,4.18,4.82,4.70,11.24,2.92,2.92,...,4.44,4.62,4.62,3.82,4.70,4.70,7.84,4.82,2.19,7.84
4,2016-09-01 04:00:00,4537.0,5.21,5.21,4.20,4.83,4.58,11.32,3.06,3.06,...,4.36,4.44,4.44,3.93,4.58,4.58,8.11,4.83,2.04,8.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52596,2022-09-01 18:00:00,3834.0,2.88,2.88,2.61,2.09,6.03,9.07,2.58,2.58,...,1.08,1.79,1.79,2.28,6.03,6.03,6.07,2.09,4.66,6.07
52597,2022-09-01 19:00:00,3645.0,2.73,2.73,2.53,1.69,6.65,9.06,2.53,2.53,...,0.17,1.08,1.08,1.82,6.65,6.65,4.97,1.69,4.21,4.97
52598,2022-09-01 20:00:00,3594.0,2.26,2.26,2.53,1.06,6.22,8.96,2.97,2.97,...,0.57,0.17,0.17,1.58,6.22,6.22,3.97,1.06,3.66,3.97
52599,2022-09-01 21:00:00,3622.0,2.09,2.09,2.91,0.89,5.82,8.81,3.82,3.82,...,0.90,0.57,0.57,2.06,5.82,5.82,3.96,0.89,3.30,3.96


Remove data from stations outside the Iberian Peninsula

In [50]:
hourly_wind_data = hourly_wind_data.drop([c for c in hourly_wind_data.columns[2:] if c not in wind_farms["farm_name"].values],axis=1)

In [51]:
hourly_wind_data

,datetime,value,A Capelada I,A Capelada II,A Pastoriza,A Ruña,Abuela Santa Ana,Abuela Santa Ana Modificación,Acampo Arias,Acampo Armijo,...,Xiabre Ampliación,Xiabre I,Xiabre II,Xunqueira,Yerga I,Yerga II,Zarzuela II,Zas,Zorraquín,Zorreras
0,2016-09-01 00:00:00,6373.0,5.37,5.37,4.41,5.19,3.18,3.18,3.34,3.34,...,5.35,5.43,5.43,4.12,5.38,5.38,5.71,5.19,2.62,5.71
1,2016-09-01 01:00:00,5976.0,5.29,5.29,4.31,5.11,3.00,3.00,3.07,3.07,...,5.03,5.35,5.35,4.06,5.29,5.29,6.51,5.11,2.49,6.51
2,2016-09-01 02:00:00,5499.0,5.31,5.31,4.24,5.05,2.94,2.94,2.48,2.48,...,4.62,5.03,5.03,4.04,4.95,4.95,7.29,5.05,2.35,7.29
3,2016-09-01 03:00:00,5031.0,5.27,5.27,4.18,4.82,2.92,2.92,1.89,1.89,...,4.44,4.62,4.62,3.82,4.70,4.70,7.84,4.82,2.19,7.84
4,2016-09-01 04:00:00,4537.0,5.21,5.21,4.20,4.83,3.06,3.06,2.06,2.06,...,4.36,4.44,4.44,3.93,4.58,4.58,8.11,4.83,2.04,8.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52596,2022-09-01 18:00:00,3834.0,2.88,2.88,2.61,2.09,2.58,2.58,5.81,5.81,...,1.08,1.79,1.79,2.28,6.03,6.03,6.07,2.09,4.66,6.07
52597,2022-09-01 19:00:00,3645.0,2.73,2.73,2.53,1.69,2.53,2.53,6.37,6.37,...,0.17,1.08,1.08,1.82,6.65,6.65,4.97,1.69,4.21,4.97
52598,2022-09-01 20:00:00,3594.0,2.26,2.26,2.53,1.06,2.97,2.97,5.10,5.10,...,0.57,0.17,0.17,1.58,6.22,6.22,3.97,1.06,3.66,3.97
52599,2022-09-01 21:00:00,3622.0,2.09,2.09,2.91,0.89,3.82,3.82,2.40,2.40,...,0.90,0.57,0.57,2.06,5.82,5.82,3.96,0.89,3.30,3.96


In [52]:
for c in hourly_wind_data.columns[2:]:
    turbines = wind_farms[wind_farms.farm_name==c].turbines.iloc[0]
    hourly_wind_data[c] = hourly_wind_data[c] * turbines

In [53]:
hourly_wind_data

,datetime,value,A Capelada I,A Capelada II,A Pastoriza,A Ruña,Abuela Santa Ana,Abuela Santa Ana Modificación,Acampo Arias,Acampo Armijo,...,Xiabre Ampliación,Xiabre I,Xiabre II,Xunqueira,Yerga I,Yerga II,Zarzuela II,Zas,Zorraquín,Zorreras
0,2016-09-01 00:00:00,6373.0,268.5,241.65,44.1,212.79,79.50,25.44,50.10,80.16,...,10.70,59.73,16.29,12.36,199.06,193.68,45.68,467.1,15.72,165.59
1,2016-09-01 01:00:00,5976.0,264.5,238.05,43.1,209.51,75.00,24.00,46.05,73.68,...,10.06,58.85,16.05,12.18,195.73,190.44,52.08,459.9,14.94,188.79
2,2016-09-01 02:00:00,5499.0,265.5,238.95,42.4,207.05,73.50,23.52,37.20,59.52,...,9.24,55.33,15.09,12.12,183.15,178.20,58.32,454.5,14.10,211.41
3,2016-09-01 03:00:00,5031.0,263.5,237.15,41.8,197.62,73.00,23.36,28.35,45.36,...,8.88,50.82,13.86,11.46,173.90,169.20,62.72,433.8,13.14,227.36
4,2016-09-01 04:00:00,4537.0,260.5,234.45,42.0,198.03,76.50,24.48,30.90,49.44,...,8.72,48.84,13.32,11.79,169.46,164.88,64.88,434.7,12.24,235.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52596,2022-09-01 18:00:00,3834.0,144.0,129.60,26.1,85.69,64.50,20.64,87.15,139.44,...,2.16,19.69,5.37,6.84,223.11,217.08,48.56,188.1,27.96,176.03
52597,2022-09-01 19:00:00,3645.0,136.5,122.85,25.3,69.29,63.25,20.24,95.55,152.88,...,0.34,11.88,3.24,5.46,246.05,239.40,39.76,152.1,25.26,144.13
52598,2022-09-01 20:00:00,3594.0,113.0,101.70,25.3,43.46,74.25,23.76,76.50,122.40,...,1.14,1.87,0.51,4.74,230.14,223.92,31.76,95.4,21.96,115.13
52599,2022-09-01 21:00:00,3622.0,104.5,94.05,29.1,36.49,95.50,30.56,36.00,57.60,...,1.80,6.27,1.71,6.18,215.34,209.52,31.68,80.1,19.80,114.84


In [54]:
for cluster in wind_farms["kmeans_labels"].unique():
    stations_in_cluster = wind_farms[wind_farms["kmeans_labels"] == cluster]["farm_name"]
    df_aux = hourly_wind_data[[c for c in hourly_wind_data.columns[2:] if c in stations_in_cluster.values]]
    print(df_aux.info())
    hourly_wind_data['mean_cluster' + str(cluster)] = df_aux.mean(axis=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52601 entries, 0 to 52600
Columns: 225 entries, A Capelada I to Zas
dtypes: float64(225)
memory usage: 90.3 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52601 entries, 0 to 52600
Columns: 174 entries, Abuela Santa Ana to Virgen de los Llanos II
dtypes: float64(174)
memory usage: 69.8 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52601 entries, 0 to 52600
Columns: 289 entries, Acampo Arias to Zorraquín
dtypes: float64(289)
memory usage: 116.0 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52601 entries, 0 to 52600
Columns: 190 entries, Alba de Tormes to Villoruebo
dtypes: float64(190)
memory usage: 76.2 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52601 entries, 0 to 52600
Columns: 121 entries, Alijar to Zorreras
dtypes: float64(121)
memory usage: 48.6 MB
None


In [55]:
hourly_wind_data

,datetime,value,A Capelada I,A Capelada II,A Pastoriza,A Ruña,Abuela Santa Ana,Abuela Santa Ana Modificación,Acampo Arias,Acampo Armijo,...,Yerga II,Zarzuela II,Zas,Zorraquín,Zorreras,mean_cluster2,mean_cluster4,mean_cluster3,mean_cluster1,mean_cluster0
0,2016-09-01 00:00:00,6373.0,268.5,241.65,44.1,212.79,79.50,25.44,50.10,80.16,...,193.68,45.68,467.1,15.72,165.59,112.185067,89.500057,79.351453,96.032947,61.524959
1,2016-09-01 01:00:00,5976.0,264.5,238.05,43.1,209.51,75.00,24.00,46.05,73.68,...,190.44,52.08,459.9,14.94,188.79,110.485689,86.846667,73.640934,97.163526,69.323058
2,2016-09-01 02:00:00,5499.0,265.5,238.95,42.4,207.05,73.50,23.52,37.20,59.52,...,178.20,58.32,454.5,14.10,211.41,108.832578,85.889023,68.388685,96.468737,74.603471
3,2016-09-01 03:00:00,5031.0,263.5,237.15,41.8,197.62,73.00,23.36,28.35,45.36,...,169.20,62.72,433.8,13.14,227.36,106.013600,84.532299,67.186090,94.921316,76.769587
4,2016-09-01 04:00:00,4537.0,260.5,234.45,42.0,198.03,76.50,24.48,30.90,49.44,...,164.88,64.88,434.7,12.24,235.19,105.079689,84.039368,70.115502,92.742579,78.005620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52596,2022-09-01 18:00:00,3834.0,144.0,129.60,26.1,85.69,64.50,20.64,87.15,139.44,...,217.08,48.56,188.1,27.96,176.03,64.667644,104.750632,98.840381,78.975263,85.693554
52597,2022-09-01 19:00:00,3645.0,136.5,122.85,25.3,69.29,63.25,20.24,95.55,152.88,...,239.40,39.76,152.1,25.26,144.13,59.437956,105.742299,106.723564,74.677947,77.808017
52598,2022-09-01 20:00:00,3594.0,113.0,101.70,25.3,43.46,74.25,23.76,76.50,122.40,...,223.92,31.76,95.4,21.96,115.13,54.616400,108.108391,110.023045,67.139737,71.382810
52599,2022-09-01 21:00:00,3622.0,104.5,94.05,29.1,36.49,95.50,30.56,36.00,57.60,...,209.52,31.68,80.1,19.80,114.84,56.218800,113.548103,114.938097,61.955158,71.923140


In [56]:
hourly_wind_data.to_csv("../data/hourly_data_with_cluster_info.csv")